<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/macro_oil_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Kernel setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
!pip install wandb
!git clone https://github.com/Krankile/npmf.git

In [3]:
%%capture
!cd npmf && git pull

In [6]:
%%capture
!pip install wandb

In [4]:
!wandb login

/bin/bash: wandb: command not found


##General setup

In [7]:
import os
from collections import defaultdict
from collections import Counter
from datetime import datetime
from operator import itemgetter

import numpy as np
from numpy.ma.core import outerproduct
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

import wandb as wb

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from npmf.utils.colors import main, main2, main3
from npmf.utils.wandb import get_dataset, put_dataset

In [8]:
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[main, main2, main3, "black"])
mpl.rcParams['figure.figsize'] = (6, 4)  # (6, 4) is default and used in the paper

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [10]:
np.random.seed(420)

# The first header 😂✨KAWAIII ^^✨ 

*   List item
*   List item



In [12]:
macro_df = get_dataset("oil-macro-data:latest", project="master-test")

wandb: Currently logged in as: kjartan (krankile). Use `wandb login --relogin` to force relogin


In [13]:
macro_df

,index,BRT-,CLc1,WTCLc1,LNG-AS,.VIX,EUR=,GBP=,CNY=
0,2000-01-03,NaN,NaN,NaN,NaN,24.21,1.0262,1.6368,8.2798
1,2000-01-04,24.40,25.55,NaN,NaN,27.01,1.0308,1.6365,8.2799
2,2000-01-05,23.65,24.91,NaN,NaN,26.41,1.0314,1.6415,8.2798
3,2000-01-06,23.54,24.78,NaN,NaN,25.73,1.0319,1.6463,8.2797
4,2000-01-07,22.95,24.22,NaN,NaN,21.72,1.0289,1.6383,8.2794
...,...,...,...,...,...,...,...,...,...
5813,2022-04-15,NaN,NaN,NaN,NaN,NaN,1.0806,1.3058,6.3705
5814,2022-04-18,NaN,108.21,108.21,NaN,22.17,1.0780,1.3008,6.3630
5815,2022-04-19,105.67,102.56,102.56,NaN,21.37,1.0786,1.2996,6.3930
5816,2022-04-20,104.78,102.75,102.19,NaN,20.32,1.0850,1.3066,6.4188


In [36]:
set(list(macro_df["index"].diff()))

#Largest gap in macro data is 4 days, which probably makes sense during holidays, etc.  

{NaT,
 Timedelta('1 days 00:00:00'),
 Timedelta('2 days 00:00:00'),
 Timedelta('3 days 00:00:00'),
 Timedelta('4 days 00:00:00')}

In [37]:
macro_df[~pd.isnull(macro_df["LNG-AS"])] #LNG-as only has 581 non NA values. Should not be detrimental to add signal. 

,index,BRT-,CLc1,WTCLc1,LNG-AS,.VIX,EUR=,GBP=,CNY=
2736,2010-07-01,72.28,72.95,72.95,6.75,32.86,1.2517,1.5173,6.7807
2741,2010-07-08,75.21,75.44,75.44,7.40,25.71,1.2693,1.5162,6.7760
2746,2010-07-15,76.27,76.62,76.62,7.75,25.14,1.2938,1.5454,6.7774
2751,2010-07-22,78.54,79.30,79.30,7.75,24.63,1.2888,1.5252,6.7795
2756,2010-07-29,79.95,78.36,78.36,8.05,24.13,1.3077,1.5609,6.7760
...,...,...,...,...,...,...,...,...,...
5793,2022-03-18,119.54,104.70,104.70,35.50,23.87,1.1055,1.3180,6.3609
5798,2022-03-25,128.91,113.90,113.90,35.00,20.81,1.0981,1.3188,6.3658
5803,2022-04-01,108.48,99.27,99.27,35.00,19.63,1.1053,1.3112,6.3625
5808,2022-04-08,100.74,98.26,98.26,33.00,21.16,1.0876,1.3031,6.3610


In [38]:
put_dataset(macro_df, filename="macro-oil-final", project="master-test")